In [5]:
import optuna
import pandas as pd
import numpy as np
import time
import os
import sys
from multiprocessing import Pool, cpu_count

# Import algorytmu Tabu Search i narzędzi pomocniczych
from src.algorithms.tabu_move import solve_tsp
from src.utils.tsp_loader import load_tsp_matrix
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat

In [6]:
# Konfiguracja eksperymentu
INSTANCES = [
    'Dane_TSP_48.xlsx',
    'Dane_TSP_76.xlsx',
    'Dane_TSP_127.xlsx'
]

# Liczba prób optymalizacji dla każdej instancji (ile razy Optuna wybierze parametry)
N_TRIALS = 100

# Liczba powtórzeń dla każdego zestawu parametrów
REPEATS = 5

In [7]:
def objective(trial, distance_matrix):
    """
    Funkcja celu dla Optuny.
    Optuna dobiera parametry, my uruchamiamy algorytm wielokrotnie (REPEATS) 
    i zwracamy wynik (minimalny koszt z powtórzeń).
    """
    # Definiujemy zakresy, z których Optuna może losować wartości.
    max_iter = trial.suggest_int("max_iter", 5_000, 100_000, step=5_000)
    stop_no_improve = trial.suggest_int("stop_no_improve", 500, 10_000, step=500)
    
    tabu_tenure = trial.suggest_int("tabu_tenure", 5, 100, step=5)
    n_neighbors = trial.suggest_int("n_neighbors", 10, 150, step=10)
    
    neighborhood_type = trial.suggest_categorical("neighborhood_type", ["swap", "insert", "two_opt"])
    
    params = {
        "max_iter": max_iter,
        "stop_no_improve": stop_no_improve,
        "tabu_tenure": tabu_tenure,
        "n_neighbors": n_neighbors,
        "neighborhood_type": neighborhood_type
    }
    
    # Uruchomienie Algorytmu Tabu Search wielokrotnie (równolegle)
    with Pool(processes=cpu_count()) as pool:
        parallel_jobs = [
            (solve_tsp, distance_matrix, params) for _ in range(REPEATS)
        ]
        # run_single_repeat zwraca (cost, route, runtime)
        results_parallel = pool.map(run_single_repeat, parallel_jobs)
    
    # Agregacja wyników
    costs = [res[0] for res in results_parallel]
    routes = [res[1] for res in results_parallel]
    runtimes = [res[2] for res in results_parallel]
    
    min_cost = min(costs)
    mean_cost = np.mean(costs)
    mean_runtime = np.mean(runtimes)
    
    # Znalezienie trasy odpowiadającej minimalnemu kosztowi
    best_idx = costs.index(min_cost)
    best_route = routes[best_idx]
    
    # Zapisanie dodatkowych statystyk w atrybutach triala
    route_str = "-".join(map(str, best_route))
    trial.set_user_attr("min_route", route_str)
    trial.set_user_attr("mean_cost", mean_cost)
    trial.set_user_attr("mean_runtime", mean_runtime)
    
    # Optuna minimalizuje wartość zwracaną. 
    # Zwracamy min_cost (najlepszy wynik z serii), aby znaleźć parametry dające szansę na najlepszy wynik.
    # Można by też zwracać mean_cost, jeśli zależy nam na stabilności.
    return min_cost

In [8]:
experiment_results = []

for instance_file in INSTANCES:
    print(f"\nOptymalizacja dla instancji: {instance_file}")
    
    distance_matrix = load_tsp_matrix(instance_file)
    
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # Parametry startowe (warm start)
    study.enqueue_trial({
        "max_iter": 5_000,
        "stop_no_improve": 500,
        "tabu_tenure": 10,
        "n_neighbors": 30,
        "neighborhood_type": "two_opt"
    })

    study.optimize(lambda trial: objective(trial, distance_matrix), n_trials=N_TRIALS)
    
    print(f"Najlepsze parametry dla {instance_file}: {study.best_params}")
    print(f"Najlepszy koszt dla {instance_file}: {study.best_value}")
    
    df_trials = study.trials_dataframe()
    
    for _, row in df_trials.iterrows():
        record = {
            "instance": instance_file,
            "max_iter": row.get("params_max_iter"),
            "tabu_tenure": row.get("params_tabu_tenure"),
            "n_neighbors": row.get("params_n_neighbors"),
            "neighborhood_type": row.get("params_neighborhood_type"),
            "stop_no_improve": row.get("params_stop_no_improve"),
            # Statystyki z powtórzeń
            "min_cost": row.get("value"),
            "mean_cost": row.get("user_attrs_mean_cost"),
            "mean_runtime": row.get("user_attrs_mean_runtime"),
            "min_route": row.get("user_attrs_min_route"),
            "state": row.get("state"),
            "trial_number": row.get("number")
        }
        experiment_results.append(record)


Optymalizacja dla instancji: Dane_TSP_48.xlsx


[I 2025-12-16 11:25:07,386] A new study created in memory with name: no-name-8ddea443-a750-4b90-a3c4-17fbca1093b4
[I 2025-12-16 11:25:17,265] Trial 0 finished with value: 9975.0 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 9975.0.
[I 2025-12-16 11:26:49,454] Trial 1 finished with value: 10769.0 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 9975.0.
[I 2025-12-16 11:26:56,171] Trial 2 finished with value: 16166.0 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 9975.0.
[I 2025-12-16 11:27:12,721] Trial 3 finished with value: 10067.0 and parameters: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}

Najlepsze parametry dla Dane_TSP_48.xlsx: {'max_iter': 10000, 'stop_no_improve': 2000, 'tabu_tenure': 5, 'n_neighbors': 50, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_48.xlsx: 9809.0

Optymalizacja dla instancji: Dane_TSP_76.xlsx


[I 2025-12-16 12:25:46,595] Trial 0 finished with value: 114537.63757013124 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 114537.63757013124.
[I 2025-12-16 12:28:15,106] Trial 1 finished with value: 127600.33964161368 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114537.63757013124.
[I 2025-12-16 12:28:28,651] Trial 2 finished with value: 231929.7125705653 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114537.63757013124.
[I 2025-12-16 12:29:01,377] Trial 3 finished with value: 98760.01745533077 and parameters: {'max_iter': 20000, 'stop_no_improve': 2000, 'tabu_tenure': 35, 'n_neighbors': 80, 'neighborhood_type': 'two_opt'}. Best is trial 3 with value: 98760

Najlepsze parametry dla Dane_TSP_76.xlsx: {'max_iter': 75000, 'stop_no_improve': 8000, 'tabu_tenure': 10, 'n_neighbors': 110, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_76.xlsx: 97728.83260645265

Optymalizacja dla instancji: Dane_TSP_127.xlsx


[I 2025-12-16 16:34:03,599] A new study created in memory with name: no-name-28a23f46-67f0-4ad9-919e-dfe0d1ff006a
[I 2025-12-16 16:34:15,200] Trial 0 finished with value: 135993.1139897495 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 135993.1139897495.
[I 2025-12-16 16:41:06,879] Trial 1 finished with value: 139732.12416098913 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 135993.1139897495.
[I 2025-12-16 16:41:22,600] Trial 2 finished with value: 253785.91294559004 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 135993.1139897495.
[I 2025-12-16 16:42:20,486] Trial 3 finished with value: 112256.88801089086 and parameters: {'max_iter': 20000, 'stop_no_improve'

Najlepsze parametry dla Dane_TSP_127.xlsx: {'max_iter': 15000, 'stop_no_improve': 10000, 'tabu_tenure': 20, 'n_neighbors': 110, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_127.xlsx: 109967.47909816584


In [9]:
if experiment_results:
    df_final = pd.DataFrame(experiment_results)
    
    # Sortowanie wyników: najpierw po instancji, potem po numerze triala
    df_final = df_final.sort_values(by=["instance", "trial_number"])
    
    save_experiment_results(df_final, time_seconds=0, subfolder="TS_Optuna", sort_by_cost=False)
    
    print("\nWyniki zostały zapisane pomyślnie w folderze project/results/TS_Optuna/")
    print(df_final[["instance", "trial_number", "min_cost", "mean_cost", "mean_runtime"]].head())


Podsumowanie (pierwsze 20 wierszy):
         instance  max_iter  tabu_tenure  n_neighbors neighborhood_type  stop_no_improve      min_cost     mean_cost  mean_runtime                                                                                                                                                                                                                                                                                                                                                                                                         min_route    state  trial_number
Dane_TSP_127.xlsx      5000           10           30           two_opt              500 135993.113990 140235.000904      4.081504 110-85-87-86-88-109-96-119-63-75-69-70-71-68-73-67-8-18-80-26-30-31-12-15-7-120-3-114-108-24-9-11-105-106-6-4-22-17-20-21-19-23-72-77-79-76-74-78-117-84-81-126-82-83-102-101-0-98-97-123-95-93-127-107-111-112-94-46-49-47-57-121-54-45-103-44-51-16-43-40-35-36-41-27-25-29-32-28-